### Pick the most optimal FPL team using Linear Programming

### ! git clone https://github.com/vaastav/Fantasy-Premier-League.git

In [1]:
import pandas as pd
from pulp import *

PATH = 'Fantasy-Premier-League/data/'

current_df = pd.read_csv(PATH + '2021-22/gws/merged_gw.csv')
current_df.tail()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
24560,Wilfred Ndidi,MID,Leicester,0.0,0,0,0,0,0.0,216,...,4,0.0,0,-202,22,224,48,True,0,38
24561,Matt Ritchie,DEF,Newcastle,0.9,0,0,3,0,0.0,292,...,1,0.0,1,143,396,253,49,False,0,38
24562,Nathan Redmond,MID,Southampton,3.4,0,0,5,0,0.0,336,...,4,0.0,2,455,683,228,59,False,0,38
24563,Mathew Ryan,GK,Brighton,0.0,0,0,0,0,0.0,65,...,3,0.0,0,-2,0,2,45,True,0,38
24564,Ryan Fredericks,DEF,West Ham,0.2,0,0,0,0,0.0,415,...,3,0.0,0,36,103,67,44,False,0,38


### Take data from a previous gameweek

In [2]:
gw_5 = current_df[current_df.GW == 5]
gw_5.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
2296,Eric Bailly,DEF,Man Utd,0.0,0,0,0,0,0.0,286,...,1,0.0,0,-624,252,876,50,False,0,5
2297,Keinan Davis,FWD,Aston Villa,0.0,0,0,0,0,0.0,49,...,3,0.0,0,-7359,9224,16583,45,True,0,5
2298,Ayotomiwa Dele-Bashiru,MID,Watford,0.0,0,0,0,0,0.0,394,...,1,0.0,0,-267,0,267,45,False,0,5
2299,James Ward-Prowse,MID,Southampton,3.0,0,0,13,1,12.2,341,...,0,0.0,3,-20295,12541,32836,64,False,0,5
2300,Bruno Miguel Borges Fernandes,MID,Man Utd,3.2,0,0,17,0,48.8,277,...,1,38.0,2,-552801,55156,607957,120,False,0,5


In [3]:
data = gw_5[['name', 'team', 'position', 'total_points', 'value']]
data.head()

,name,team,position,total_points,value
2296,Eric Bailly,Man Utd,DEF,0,50
2297,Keinan Davis,Aston Villa,FWD,0,45
2298,Ayotomiwa Dele-Bashiru,Watford,MID,0,45
2299,James Ward-Prowse,Southampton,MID,3,64
2300,Bruno Miguel Borges Fernandes,Man Utd,MID,2,120


In [4]:
# Helper variables
POS = data.position.unique()
CLUBS = data.team.unique()
BUDGET = 1000
pos_available = {
    'DEF': 5,
    'FWD': 3,
    'MID': 5,
    'GK': 2,
}

# Initialize Variables
names = [data.name[i] for i in data.index]
teams = [data.team[i] for i in data.index]
positions = [data.position[i] for i in data.index]
prices = [data.value[i] for i in data.index]
points = [data.total_points[i] for i in data.index]
players = [LpVariable("player_" + str(i), cat="Binary") for i in data.index]

In [5]:
# Initialize the problem
prob = LpProblem("FPL Player Choices", LpMaximize)

C:\Users\sitholeb\AppData\Roaming\Python\Python310\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [6]:
# Define the objective
prob += lpSum(players[i] * points[i] for i in range(len(data))) # Objective

In [7]:
# Build the constraints
prob += lpSum(players[i] * data.value[data.index[i]] for i in range(len(data))) <= BUDGET # Budget Limit

for pos in POS:
  prob += lpSum(players[i] for i in range(len(data)) if positions[i] == pos) <= pos_available[pos] # Position Limit

for club in CLUBS:
  prob += lpSum(players[i] for i in range(len(data)) if teams[i] == club) <= 3 # Club Limit

In [8]:
# Solve the problem
prob.solve()

1

In [9]:
for v in prob.variables():
  if v.varValue != 0:
    name = data.name[int(v.name.split("_")[1])]
    club = data.team[int(v.name.split("_")[1])]
    position = data.position[int(v.name.split("_")[1])]
    point = data.total_points[int(v.name.split("_")[1])]
    price = data.value[int(v.name.split("_")[1])]
    print(name, position, club, point, price, sep=" | ")

Joshua King | FWD | Watford | 10 | 55
Aaron Ramsdale | GK | Arsenal | 9 | 45
Mohamed Salah | MID | Liverpool | 12 | 125
Virgil van Dijk | DEF | Liverpool | 12 | 65
Saïd Benrahma | MID | West Ham | 10 | 64
Antonio Rüdiger | DEF | Chelsea | 14 | 55
Ismaila Sarr | MID | Watford | 15 | 60
Marcos Alonso | DEF | Chelsea | 10 | 57
Matthew Cash | DEF | Aston Villa | 15 | 50
Ivan Toney | FWD | Brentford | 12 | 63
David de Gea | GK | Man Utd | 10 | 50
Danny Welbeck | FWD | Brighton | 9 | 60
Martin Ødegaard | MID | Arsenal | 11 | 55
Leon Bailey | MID | Aston Villa | 9 | 64
Thiago Emiliano da Silva | DEF | Chelsea | 15 | 54


In [10]:
score = str(prob.objective)
constraint = [str(const) for const in prob.constraints.values()][0]
for v in prob.variables():
  score = score.replace(v.name, str(v.varValue))
  constraint = constraint.replace(v.name, str(v.varValue))

score_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', score) )
constraint_pretty = " + ".join( re.findall('[0-9\.]*\*1.0', constraint) )

print("Constraint: ")
print(constraint_pretty + " = " + str(eval(constraint_pretty)))
print()
print("Score: ")
print(score_pretty + " = " + str(eval(score_pretty)))

Constraint: 
55*1.0 + 45*1.0 + 125*1.0 + 65*1.0 + 64*1.0 + 55*1.0 + 60*1.0 + 57*1.0 + 50*1.0 + 63*1.0 + 50*1.0 + 60*1.0 + 55*1.0 + 64*1.0 + 54*1.0 = 922.0

Score: 
10*1.0 + 9*1.0 + 12*1.0 + 12*1.0 + 10*1.0 + 14*1.0 + 15*1.0 + 10*1.0 + 15*1.0 + 12*1.0 + 10*1.0 + 9*1.0 + 11*1.0 + 9*1.0 + 15*1.0 = 173.0
